Vamos a comenzarr con la creación de la base de datos y el Análisis Exploratorio de Datos

Lo primero que haremos será la automatización de la descarga de los datos

primero los de calidad del aire

In [ ]:
import os
import requests
import zipfile
from io import BytesIO

input_folder_aire = "/content/sample_data/Calidad_del_aire_original"
input_folder_meteoro='/content/sample_data/Meteorología_Original'

os.makedirs(input_folder_aire, exist_ok=True)
os.makedirs(input_folder_meteoro, exist_ok=True)

urls = {
    2024: "https://datos.madrid.es/egob/catalogo/201200-10306320-calidad-aire-horario.zip",
    2023: "https://datos.madrid.es/egob/catalogo/201200-10306319-calidad-aire-horario.zip",
    2022: "https://datos.madrid.es/egob/catalogo/201200-10306318-calidad-aire-horario.zip",
    2021: "https://datos.madrid.es/egob/catalogo/201200-10306317-calidad-aire-horario.zip",
    2020: "https://datos.madrid.es/egob/catalogo/201200-10306316-calidad-aire-horario.zip",
    2019: "https://datos.madrid.es/egob/catalogo/201200-42-calidad-aire-horario.zip"
}

for anio, url in urls.items():
    print(f"Descargando {anio}...")
    response = requests.get(url)
    response.raise_for_status()
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        for file_info in z.infolist():
            if file_info.filename.lower().endswith(".csv"):
               filename = os.path.basename(file_info.filename)
               with z.open(file_info) as source, open(os.path.join(input_folder_aire, filename), 'wb') as target:
                  target.write(source.read())
    print(f"{anio} descargado y extraído.")



No he automatizado la descarga de los archivos de meteorología debido a que las urls de los csv no siguen un patrón constante y supondría poner manualmente las urls de cada mes, se pueden descargar aquí: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=fa8357cec5efa610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default , descargar solo desde 2019 hasta 2024y añadirlos a la carpeta /content/sample_data/Meteorología_Original

Lo primero que vamos a realizar es la conversión de las horas de columnas a filas para los datos de calidad del aire

In [ ]:
import pandas as pd
import os
from glob import glob


input_folder = "/content/sample_data/Calidad_del_aire_original"
output_folder = "/content/sample_data/Calidad_del_aire_1_transformacion"


os.makedirs(output_folder, exist_ok=True)

csv_files = glob(os.path.join(input_folder, "*.csv"))

for file_path in csv_files:
  print(f"Procesando archivo: {os.path.basename(file_path)}")
  df = pd.read_csv(file_path, sep=";", encoding="latin1")
  rows = []
  for _, row in df.iterrows():
      base_data = row[['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD',
                      'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA']]
      for hour in range(1, 25):
          h_col = f'H{hour:02d}'
          v_col = f'V{hour:02d}'
          new_row = {
              'PROVINCIA': base_data['PROVINCIA'],
              'MUNICIPIO': base_data['MUNICIPIO'],
              'ESTACION': base_data['ESTACION'],
              'MAGNITUD': base_data['MAGNITUD'],
              'PUNTO_MUESTREO': base_data['PUNTO_MUESTREO'],
              'ANO': base_data['ANO'],
              'MES': base_data['MES'],
              'DIA': base_data['DIA'],
              'HORA': hour,
              'VALOR_HORA': row[h_col],
              'VALIDACION': row[v_col]
          }
          rows.append(new_row)
  df_transformado = pd.DataFrame(rows)
  output_file = os.path.join(
    output_folder, os.path.basename(file_path).replace(".csv", "_transformado.csv"))
  df_transformado.to_csv(output_file, index=False)
print("Transformación completada para todos los archivos.")

Haremos ahora lo mismo para los csv de meteorología

In [ ]:
input_folder='/content/sample_data/Meteorología_Original'
output_folder='/content/sample_data/Meteorología_1_transformacion'

os.makedirs(output_folder, exist_ok=True)
csv_files = glob(os.path.join(input_folder, "*.csv"))

for file_path in csv_files:
    print(f"Procesando archivo: {os.path.basename(file_path)}")

    df = pd.read_csv(file_path, sep=";", encoding="latin1")


    rows = []
    for _, row in df.iterrows():
        base_data = row[['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD',
                         'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA']]
        for hour in range(1, 25):
            h_col = f'H{hour:02d}'
            v_col = f'V{hour:02d}'
            new_row = {
                'PROVINCIA': base_data['PROVINCIA'],
                'MUNICIPIO': base_data['MUNICIPIO'],
                'ESTACION': base_data['ESTACION'],
                'MAGNITUD': base_data['MAGNITUD'],
                'PUNTO_MUESTREO': base_data['PUNTO_MUESTREO'],
                'ANO': base_data['ANO'],
                'MES': base_data['MES'],
                'DIA': base_data['DIA'],
                'HORA': hour,
                'VALOR_HORA': row[h_col],
                'VALIDACION': row[v_col]
            }
            rows.append(new_row)

    df_transformado = pd.DataFrame(rows)

    output_file = os.path.join(
        output_folder, os.path.basename(file_path).replace(".csv", "_transformado.csv"))
    df_transformado.to_csv(output_file, index=False)

print("Transformación completada para todos los archivos de meteorología.")

Vale, visualizaremos los datos, para ver que tenemos las mismas columnas en los dos, cojeremos archivos aleatorios de cada carpeta ya que al haber sido transformados con el mismo método,si uno está bien, lo estarán todos

In [ ]:
carpeta_aire = "/content/sample_data/Calidad_del_aire_1_transformacion"
carpeta_meteo = "/content/sample_data/Meteorología_1_transformacion"
import random
archivo_aire = random.choice(glob(os.path.join(carpeta_aire, "*.csv")))
archivo_meteo = random.choice(glob(os.path.join(carpeta_meteo, "*.csv")))

df_aire = pd.read_csv(archivo_aire)
df_meteo = pd.read_csv(archivo_meteo)

print(f"Archivo de calidad del aire cargado: {os.path.basename(archivo_aire)}")
print(df_aire.info())

print(f"Archivo de meteorología cargado: {os.path.basename(archivo_meteo)}")
print(df_meteo.info())

Vale, ahora hemos visto que tienen 11 columnas los dos entonces está correcto, hemos visto también los tipos de dato de cada columna.



Ahora vamos a plantear el ejercicio, una vez tenemos ya las estructuras principales organizadas, lo que vamos a querer hacer va a ser lo siguiente, queremos que la estructura final de nuestra base de datos, sea de las columnas ESTACION_calidad,MAGNITUD,ANO,MES,DIA,HORA,VALOR_HORA,VALOR_HORA_80, VALOR_HORA_81,VALOR_HORA_82,VALOR_HORA_83,VALOR_HORA_86,VALOR_HORA_87,VALOR_HORA_88,VALOR_HORA_89, siendo VALOR_HORA la variable objetivo, lo que haremos una vez tengamos esto es hacer one-hot encoding en las variables estacion y magnitud, así no asignará mas importancia a una magnitud por tener un número mayor, simplemente las organizará asignandole un 1 a cada una poniéndolas en columnas, con año, mes, dia y hora, las haremos variables cíclicas usando senos y cosenos, por último normalizaremos los datos para que vayan todos de 0 a 1 en función de su magnitud y columna.

Siguiente paso pasar los csv de meteorología a PROVINCIA	MUNICIPIO	ESTACION	ANO	MES	DIA	HORA	VALOR_HORA_80	VALOR_HORA_81	VALOR_HORA_82	VALOR_HORA_83	VALOR_HORA_86	VALOR_HORA_87	VALOR_HORA_88	VALOR_HORA_89

In [ ]:
input_folder = '/content/sample_data/Meteorología_1_transformacion'
output_folder = '/content/sample_data/Meteorología_2_transformacion'
os.makedirs(output_folder, exist_ok=True)
csv_files = glob(os.path.join(input_folder, "*.csv"))
for file_path in csv_files:
    print(f"Procesando {os.path.basename(file_path)}...")

    df = pd.read_csv(file_path)

    df_pivot = df.pivot_table(
        index=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'ANO', 'MES', 'DIA', 'HORA'],
        columns='MAGNITUD',
        values='VALOR_HORA'
    ).reset_index()
    df_pivot.columns.name = None
    df_pivot = df_pivot.rename(columns={m: f'VALOR_HORA_{int(m)}' for m in df_pivot.columns if isinstance(m, (int, float))})

    output_file = os.path.join(output_folder, os.path.basename(file_path).replace(".csv", "2_transformacion.csv"))
    df_pivot.to_csv(output_file, index=False)


print("Todos los archivos han sido transformados y guardados.")



Comprobemos que está todo en orden

In [ ]:
archivo_random_nueva_tranformacion=random.choice(glob(os.path.join('/content/sample_data/Meteorología_2_transformacion', "*.csv")))
df = pd.read_csv(archivo_random_nueva_tranformacion)
print(f"Examinando el siguiente archivo {os.path.basename(archivo_random_nueva_tranformacion)}...")
df.info()
df.head()

Vemos que la magnitud 80 no la está cogiendo, vamos a ver porque, para ello, examinaremos todos los cvs de la carpeta antes de transformar y ver que archivos si que tenían en la columna magnitud, el valor 80

In [ ]:
folder_path = "/content/sample_data/Meteorología_1_transformacion"

csv_files = glob(os.path.join(folder_path, "*.csv"))

total_archivos = 0
archivos_con_magnitud_80 = 0

for file_path in csv_files:
    total_archivos += 1
    try:
        df = pd.read_csv(file_path)
        if 'MAGNITUD' in df.columns and (df['MAGNITUD'] == 80).any():
            archivos_con_magnitud_80 += 1
    except Exception as e:
        print(f"Error leyendo {file_path}: {e}")

print(f"Total de archivos analizados: {total_archivos}")
print(f"Archivos que contienen MAGNITUD 80: {archivos_con_magnitud_80}")

Vemos que solo hay 5 archivos que tienen magnitud 80, ahora sería interesante comprobar cuanta cantidasd de valores de magnitud 80 tenemso en esos 5 archivos comparados con la cantidad de informacion que tenemos de otras magnitudes.

In [ ]:
conteo_total = {}

for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        if 'MAGNITUD' in df.columns:
            for magnitud in df['MAGNITUD'].unique():
                count = len(df[df['MAGNITUD'] == magnitud])
                conteo_total[magnitud] = conteo_total.get(magnitud, 0) + count
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")

print("Conteo de registros por MAGNITUD:")
for magnitud, count in sorted(conteo_total.items()):
    print(f"MAGNITUD {int(magnitud)}: {count} registros")

Vemos que comparado con la cantidad de valores nulos que nos va a dar la magnitud 80, no nos vale la pena tenerlo en cuenta, es una decisión que se ha tomado
durante el procesamiento de los datos, así que vamos a cargarnos las columnas de valor_80 que tengamos en los archivos transformados

In [ ]:
folder_path = "/content/sample_data/Meteorología_2_transformacion"
csv_files = glob(os.path.join(folder_path, "*.csv"))

archivos_con_valor_80 = 0

for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)

        if 'VALOR_HORA_80' in df.columns:
            archivos_con_valor_80 += 1
            df.drop(columns=['VALOR_HORA_80'], inplace=True)
            df.to_csv(file_path, index=False)

    except Exception as e:
        print(f"Error en {file_path}: {e}")

print(f"Total de archivos analizados: {len(csv_files)}")
print(f"Archivos que tenían la columna 'VALOR_HORA_80' y fueron modificados: {archivos_con_valor_80}")

Ahora contaremos cuantos csv tienen la columna VALOR_HORA_80 para asegurarnos


In [ ]:
valor_80=0
for file_path in csv_files:
        df = pd.read_csv(file_path)
        if 'VALOR_HORA_80' in df.columns:
            archivos_con_valor_80 += 1
print(archivos_con_valor_80)

Genial, ya está completado este proceso, como queremos combinar los df de meteorología y de calidad del aire, tendremos que ver que estaciones coinciden y cuales están cercas de otras
para así poder asociar los datos de calidad del aire con los datos de meteorología. Automatizamos la descarga

In [ ]:

carpeta_estaciones='/content/sample_data/Estaciones'
os.makedirs(carpeta_estaciones, exist_ok=True)

archivos = {
    "https://datos.madrid.es/egob/catalogo/212629-1-estaciones-control-aire.csv":
        os.path.join(carpeta_estaciones, "informacion_estaciones_red_calidad_aire.csv"),

    "https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv":
        os.path.join(carpeta_estaciones, "Estaciones_control_datos_meteorologicos.csv")
}
for url, destino in archivos.items():
    print(f"Descargando {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(destino, 'wb') as f:
            f.write(response.content)
        print(f"Guardado en {destino}")
    except Exception as e:
        print(f"Error al descargar {url}: {e}")
file_estaciones_meteo = '/content/sample_data/Estaciones/Estaciones_control_datos_meteorologicos.csv'
file_estaciones_aire = '/content/sample_data/Estaciones/informacion_estaciones_red_calidad_aire.csv'
df_estaciones_meteo = pd.read_csv(file_estaciones_meteo, sep=";")
df_estaciones_aire = pd.read_csv(file_estaciones_aire, sep=";")
print(df_estaciones_meteo.columns.tolist())
print(df_estaciones_aire.columns.tolist())

Veamos cuantas estaciones tienen cada uno

In [ ]:
num_estaciones_meteo = df_estaciones_meteo['ESTACION'].nunique()
num_estaciones_aire = df_estaciones_aire['ESTACION'].nunique()

print(f"Estaciones meteorológicas únicas: {num_estaciones_meteo}")
print(f"Estaciones de calidad del aire únicas: {num_estaciones_aire}")

Vemos que hay mas estaciones de meteorología, pero entre las que no coincidan, nos vamos a quedar con las estaciones de calidad del aire, ya que es el df que contiene la variable objetivo, por lo tanto entre las que no coincidan, veremos cual es la estación de meteorología que está mas cerca y se la asociaremos

Ver las que coinciden

In [ ]:
df_estaciones_meteo = df_estaciones_meteo.rename(columns={'CÓDIGO_CORTO': 'CODIGO_CORTO'})
coincidentes = pd.merge(
    df_estaciones_meteo,
    df_estaciones_aire,
    on='CODIGO_CORTO',
    suffixes=('_meteo', '_aire')
)
print("Estaciones que coinciden por CODIGO_CORTO:")
print(coincidentes[['CODIGO_CORTO', 'ESTACION_meteo', 'ESTACION_aire']])

Crearemos un mapa con folium (visto en clase) para poner en el mapa las estaciones meterológicas de un color y las de calidad del aire de otro color

In [ ]:
import folium
mapa = folium.Map(location=[40.42, -3.70], zoom_start=11)

for _, row in df_estaciones_meteo.iterrows():
    folium.CircleMarker(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=f"Meteo: {row['ESTACION']}"
    ).add_to(mapa)

for _, row in df_estaciones_aire.iterrows():
    folium.CircleMarker(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=f"Aire: {row['ESTACION']}"
    ).add_to(mapa)


os.makedirs('/content/MAPA', exist_ok=True)

mapa.save("/content/MAPA/mapa.html")
print("Mapa generado: mapa.html")

Veamos las que están cerca

In [ ]:
from geopy.distance import geodesic


codigos_coincidentes = set(coincidentes['CODIGO_CORTO'])


aire_sin_pareja = df_estaciones_aire[~df_estaciones_aire['CODIGO_CORTO'].isin(codigos_coincidentes)]


candidatas_meteo = df_estaciones_meteo.copy()

asociaciones = []

for _, aire in aire_sin_pareja.iterrows():
    punto_aire = (aire['LATITUD'], aire['LONGITUD'])
    distancia_min = float('inf')
    estacion_meteo_cercana = None
    codigo_meteo_cercano = None

    for _, meteo in candidatas_meteo.iterrows():
        punto_meteo = (meteo['LATITUD'], meteo['LONGITUD'])
        dist = geodesic(punto_aire, punto_meteo).meters
        if dist < distancia_min:
            distancia_min = dist
            estacion_meteo_cercana = meteo['ESTACION']
            codigo_meteo_cercano = meteo['CODIGO_CORTO']

    asociaciones.append({
        "CODIGO_CORTO_AIRE": aire['CODIGO_CORTO'],
        "ESTACION_aire": aire['ESTACION'],
        "CODIGO_CORTO_METEO_MAS_CERCANA": codigo_meteo_cercano,
        "ESTACION_meteo_mas_cercana": estacion_meteo_cercana,
        "DISTANCIA_M": round(distancia_min, 2)
    })


df_asociaciones = pd.DataFrame(asociaciones)
df_asociaciones_sorted = df_asociaciones.sort_index()
num=df_asociaciones_sorted.count()
df_asociaciones_sorted.head(num[0])

Genial, ya tenemos las más cercanas, crearemos un diccionario

In [ ]:
equivalencias = {
    str(row['CODIGO_CORTO']).zfill(3): str(row['CODIGO_CORTO']).zfill(3)
    for _, row in coincidentes.iterrows()
}

for _, row in df_asociaciones.iterrows():
    equivalencias[str(row['CODIGO_CORTO_AIRE']).zfill(3)] = str(row['CODIGO_CORTO_METEO_MAS_CERCANA']).zfill(3)

print(equivalencias)

Ahora borraremos las columnas que no aportan valor añadido de los csv, van a ser PROVINCIA, MUNICIPIO y PUNTO_MUESTREO

In [ ]:
columnas_a_eliminar = ['PROVINCIA', 'MUNICIPIO', 'PUNTO_MUESTREO']

carpetas = {
    "/content/sample_data/Calidad_del_aire_1_transformacion": "/content/sample_data/Calidad_del_aire_2_transformacion"
    "/content/sample_data/Meteorología_2_transformacion": "/content/sample_data/Meteorología_3_transformacion"
}

for carpeta_salida in carpetas.values():
    os.makedirs(carpeta_salida, exist_ok=True)

for carpeta_entrada, carpeta_salida in carpetas.items():
    for nombre_archivo in os.listdir(carpeta_entrada):
        if nombre_archivo.endswith(".csv"):
            ruta_entrada = os.path.join(carpeta_entrada, nombre_archivo)
            df = pd.read_csv(ruta_entrada)
            df = df.drop(columns=columnas_a_eliminar, errors='ignore')
            nuevo_nombre = nombre_archivo.replace(".csv", "3.csv")
            ruta_salida = os.path.join(carpeta_salida, nuevo_nombre)
            df.to_csv(ruta_salida, index=False)
            print(f"Guardado: {ruta_salida}")



Comprobemos que esta todo en orden

In [ ]:

for nombre_carpeta, ruta in carpetas.items():
    archivos = [f for f in os.listdir(ruta) if f.endswith(".csv")]
    if archivos:
        archivo_aleatorio = random.choice(archivos)
        ruta_archivo = os.path.join(ruta, archivo_aleatorio)
        print(f"\nCarpeta: {nombre_carpeta}")
        print(f"Archivo seleccionado: {archivo_aleatorio}")
        df = pd.read_csv(ruta_archivo)
        print("\nColumnas y tipos de dato:")
        print(df.dtypes)
    else:
        print(f"No se encontraron archivos en la carpeta: {nombre_carpeta}")

Genial, vamos a ver ahora cuantos valores de VALOR_HORA de calidad del aire estan validados

In [ ]:
carpeta = "/content/sample_data/Calidad_del_aire_2_transformacion"

total_v = 0
total_n = 0
for archivo in os.listdir(carpeta):
    if archivo.endswith(".csv"):
        ruta = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta)
        if 'VALIDACION' in df.columns:
            total_v += (df['VALIDACION'] == 'V').sum()
            total_n += (df['VALIDACION'] == 'N').sum()

print(f"Total 'V': {total_v}")
print(f"Total 'N': {total_n}")

Los valores sin validar constituyen menos del 2 por ciento de los datos, la decisión es eliminarlos para mejorar la calidad del Dataset

In [ ]:
carpeta_entrada = "/content/sample_data/Calidad_del_aire_2_transformacion"
carpeta_salida = "/content/sample_data/Calidad_del_aire_3_transformacion"

os.makedirs(carpeta_salida, exist_ok=True)

for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith(".csv"):
        ruta_entrada = os.path.join(carpeta_entrada, archivo)
        try:
            df = pd.read_csv(ruta_entrada)
            if 'VALIDACION' in df.columns:
                df = df[df['VALIDACION'] == 'V']
            nuevo_nombre = archivo.replace(".csv", "4.csv")
            ruta_salida = os.path.join(carpeta_salida, nuevo_nombre)
            df.to_csv(ruta_salida, index=False)
            print(f"Guardado sin 'N': {ruta_salida}")
        except Exception as e:
            print(f"Error procesando {archivo}: {e}")

Comprobemos

In [ ]:
carpeta = "/content/sample_data/Calidad_del_aire_3_transformacion"

total_v = 0
total_n = 0
for archivo in os.listdir(carpeta):
    if archivo.endswith(".csv"):
        ruta = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta)
        if 'VALIDACION' in df.columns:
            total_v += (df['VALIDACION'] == 'V').sum()
            total_n += (df['VALIDACION'] == 'N').sum()

print(f"Total 'V': {total_v}")
print(f"Total 'N': {total_n}")

Ahora que ya tenemos solo V en la columna VALIDACION, la eliminamos ya que ya no nos aporta valor

In [ ]:
carpeta_entrada = "/content/sample_data/Calidad_del_aire_3_transformacion"
carpeta_salida = "/content/sample_data/Calidad_del_aire_4_transformacion"

os.makedirs(carpeta_salida, exist_ok=True)

for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith(".csv"):
        ruta_entrada = os.path.join(carpeta_entrada, archivo)
        try:
            df = pd.read_csv(ruta_entrada)
            df = df.drop(columns=['VALIDACION'], errors='ignore')
            nuevo_nombre = archivo.replace(".csv", "5.csv")
            ruta_salida = os.path.join(carpeta_salida, nuevo_nombre)
            df.to_csv(ruta_salida, index=False)
            print(f"VALIDACION eliminada y guardado: {ruta_salida}")
        except Exception as e:
            print(f"Error procesando {archivo}: {e}")

Comprobacion

In [ ]:
carpeta = "/content/sample_data/Calidad_del_aire_4_transformacion"
archivos = [f for f in os.listdir(carpeta) if f.endswith(".csv")]
archivo_random = random.choice(archivos)
ruta_archivo = os.path.join(carpeta, archivo_random)
df = pd.read_csv(ruta_archivo)
print(df.columns.tolist())

Genial, los datasets ya están preparados para juntarlos de forma que quede de la siguiente manera 'ESTACION_CA'(ya que es la estacion de calidad del aire la que se va a a mantener en el dataset), 'MAGNITUD', 'ANO', 'MES', 'DIA', 'HORA', 'VALOR_HORA''VALOR_HORA_81''VALOR_HORA_82''VALOR_HORA_83''VALOR_HORA_86' 'VALOR_HORA_87''VALOR_HORA_88''VALOR_HORA_89'

Pero antes de juntarlos, voy a rellenar los valores nulos que hay en el dataset meteorológico, porque lo estaba intentando rellenar cuando ya estaba todo en el mismo dataframe, y es peor, ya que requiere de mucho más capacidad y de tiempo, entonces, lo que voy a hacer es los valores de magnitudes que sean nulos, sustituirlos por la media de la estación más cercana para esa magnitud, para ello haremos un dataframe de estaciones cercanas

In [ ]:
from geopy.distance import geodesic
import os

df_aire = pd.read_csv("/content/sample_data/Estaciones/informacion_estaciones_red_calidad_aire.csv", sep=";")
df_meteo = pd.read_csv("/content/sample_data/Estaciones/Estaciones_control_datos_meteorologicos.csv", sep=";")


df_aire['CODIGO_CORTO'] = df_aire['CODIGO_CORTO'].astype(str).str.zfill(3)
df_meteo['CÓDIGO_CORTO'] = df_meteo['CÓDIGO_CORTO'].astype(str).str.zfill(3)


estaciones_aire = df_aire[['CODIGO_CORTO', 'LATITUD', 'LONGITUD']].dropna()
estaciones_meteo = df_meteo[['CÓDIGO_CORTO', 'LATITUD', 'LONGITUD']].dropna()

distancias = []
for _, est_a in estaciones_aire.iterrows():
    for _, est_m in estaciones_meteo.iterrows():
        punto_a = (est_a['LATITUD'], est_a['LONGITUD'])
        punto_m = (est_m['LATITUD'], est_m['LONGITUD'])
        dist = geodesic(punto_a, punto_m).kilometers
        distancias.append({
            'ESTACION_CA': est_a['CODIGO_CORTO'],
            'ESTACION_METEO': est_m['CÓDIGO_CORTO'],
            'DISTANCIA_KM': dist
        })

df_dist = pd.DataFrame(distancias)
df_dist = df_dist.sort_values(['ESTACION_CA', 'DISTANCIA_KM'])

output_path = "/content/sample_data/Estaciones/equivalencias_estaciones_ordenadas.csv"
df_dist.to_csv(output_path, index=False)
print(f"Distancias guardadas en: {output_path}")

Ahora realizaremos el relleno de los valores nulos de las columnas meteorológicas

Hace lo siguiente:

Para cada NaN, busca una estación cercana equivalente en la tabla de equivalencias.

Intenta rellenar ese NaN con la media mensual de esa magnitud en la estación meteorológica cercana.

Si encuentra una media válida (no NaN), la usa para rellenar el valor

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


carpeta_origen = r"/content/sample_data/Calidad_del_aire_4_transformacion"
carpeta_salida = r"/content/sample_data/Calidad_del_aire_4_transformacion_Relleno"
ruta_equivalencias = r"/content/sample_data/Estaciones/equivalencias_estaciones_ordenadas.csv"


os.makedirs(carpeta_salida, exist_ok=True)


df_dist = pd.read_csv(ruta_equivalencias, dtype=str)
df_dist['ESTACION_CA'] = df_dist['ESTACION_CA'].str.zfill(3)
df_dist['ESTACION_METEO'] = df_dist['ESTACION_METEO'].str.zfill(3)


archivos = [f for f in os.listdir(carpeta_origen) if f.endswith(".csv")]


for archivo in tqdm(archivos, desc="Rellenando"):
    df = pd.read_csv(os.path.join(carpeta_origen, archivo))
    df['ESTACION'] = df['ESTACION'].astype(str).str.zfill(3)

    columnas_meteo = [col for col in df.columns if col.startswith("VALOR_HORA_8")]

    for col in columnas_meteo:
        nulos = df[col].isna()
        for idx in df[nulos].index:
            row = df.loc[idx]
            est_origen = row['ESTACION']
            y, m = row['ANO'], row['MES']

            estaciones_cercanas = df_dist[df_dist['ESTACION_CA'] == est_origen]['ESTACION_METEO'].tolist()
            for est_cercana in estaciones_cercanas:
                media = df[
                    (df['ESTACION'] == est_cercana) &
                    (df['ANO'] == y) &
                    (df['MES'] == m)
                ][col].mean(skipna=True)

                if not np.isnan(media):
                    df.at[idx, col] = media
                    break


    nuevo_nombre = archivo.replace(".csv", "5.csv")
    ruta_salida = os.path.join(carpeta_salida, nuevo_nombre)
    df.to_csv(ruta_salida, index=False)


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


carpeta_origen = r"/content/sample_data/Calidad_del_aire_4_transformacion_Relleno"
carpeta_salida = r"/content/sample_data/Meteo_Transformado:Final"
os.makedirs(carpeta_salida, exist_ok=True)


columnas_meteo = [f"VALOR_HORA_8{i}" for i in [1,2,3,6,7,8,9]]

archivos = [f for f in os.listdir(carpeta_origen) if f.endswith(".csv")]

for archivo in tqdm(archivos, desc="Rellenando con media mensual"):
    df = pd.read_csv(os.path.join(carpeta_origen, archivo))

    for col in columnas_meteo:
        if col not in df.columns:
            continue

     
        medias_mensuales = df.groupby(['ANO', 'MES'])[col].transform('mean')
        df[col] = df[col].fillna(medias_mensuales)

    nuevo_nombre = archivo.replace(".csv", "_final.csv")
    df.to_csv(os.path.join(carpeta_salida, nuevo_nombre), index=False)


In [ ]:
ruta_drive = "/content/drive/MyDrive/Mis_Datasets/MeteoFinal"
ruta_colab = "/content/sample_data/Meteo_Transformado_Final"

shutil.copytree(ruta_drive, ruta_colab, dirs_exist_ok=True)
print("Carpeta copiada a Colab.")

Vale, ahora ya tenemos los nuevos dato en la carpeta /content/sample_data/Meteo_Transformado_Final, ahora veamos la consistencia de los datos, contaremos los nulos y compararemos la media de las magnitudes cada mes antes de rellenar nulos y después (asi comprobaremos que los datos no han dado un cambio brusco)

In [ ]:
import os
import pandas as pd
from collections import defaultdict


carpeta_original = "/content/sample_data/Meteorología_3_transformacion"
carpeta_relleno = "/content/sample_data/Meteo_Transformado_Final"


nulos_totales = defaultdict(int)
medias_antes = defaultdict(list)
medias_despues = defaultdict(list)


for archivo in os.listdir(carpeta_original):
    if archivo.endswith(".csv"):
        archivo_relleno = archivo.replace(".csv", "5_final.csv")
        ruta_ori = os.path.join(carpeta_original, archivo)
        ruta_new = os.path.join(carpeta_relleno, archivo_relleno)

        if not os.path.exists(ruta_new):
            print(f"Archivo faltante: {archivo_relleno}")
            continue

        df_ori = pd.read_csv(ruta_ori)
        df_new = pd.read_csv(ruta_new)

        columnas_meteo = [col for col in df_new.columns if col.startswith("VALOR_HORA_8")]

        for col in columnas_meteo:
            nulos_totales[col] += df_new[col].isna().sum()
            medias_antes[col].append(df_ori[col].mean(skipna=True))
            medias_despues[col].append(df_new[col].mean(skipna=True))


print("Nulos por columna (en archivos con relleno):")
for col in sorted(nulos_totales):
    print(f"{col}: {nulos_totales[col]}")

print("\nMedia total antes vs después de la imputación:")
for col in sorted(medias_antes):
    media_a = pd.Series(medias_antes[col]).mean()
    media_d = pd.Series(medias_despues[col]).mean()
    print(f"{col}: antes = {media_a:.2f} | después = {media_d:.2f}")



Vemos que ya no hay ningún nulo y en aspectos generales, mantiene una buena consistencia de la media a pesar de haber rellenado muchos valores (esto se debe a que el método de elegir la estación mas cercana tenía consistencia y sentido según el problema)

Ahora, vamos a combinar los datos, de forma que por cada valor_hora que será la variable objetivo en nuestro modelo, tenga variables de magnitudes meteorológicas, que han sido rellenadas previamente las que eran nulas con un lógica y de esta forma cada valor_hora tendra datos de todas las magnitudes meteorológicas

In [ ]:
from tqdm import tqdm
carpeta_calidad = "/content/sample_data/Calidad_del_aire_4_transformacion"
carpeta_meteo = "/content/sample_data/Meteo_Transformado_Final"

df_meteo_total = pd.concat(
    [pd.read_csv(os.path.join(carpeta_meteo, f)) for f in os.listdir(carpeta_meteo) if f.endswith(".csv")],
    ignore_index=True
)
df_meteo_total['ESTACION'] = df_meteo_total['ESTACION'].astype(str).str.zfill(3)


datasets_combinados = []
contador_uniones = 0


archivos_ca = [f for f in os.listdir(carpeta_calidad) if f.endswith(".csv")]

for archivo_ca in tqdm(archivos_ca, desc="Combinando datos"):
    df_ca = pd.read_csv(os.path.join(carpeta_calidad, archivo_ca))
    df_ca['ESTACION'] = df_ca['ESTACION'].astype(str).str.zfill(3)

    estaciones_ca = df_ca['ESTACION'].unique()

    for estacion_ca in estaciones_ca:
        if estacion_ca in equivalencias:
            estacion_meteo = equivalencias[estacion_ca]

            df_ca_filtrada = df_ca[df_ca['ESTACION'] == estacion_ca]
            df_meteo_filtrada = df_meteo_total[df_meteo_total['ESTACION'] == estacion_meteo]

            df_merged = pd.merge(
                df_ca_filtrada,
                df_meteo_filtrada,
                on=['ANO', 'MES', 'DIA', 'HORA'],
                suffixes=('', '_meteo')
            )

            if not df_merged.empty:
                contador_uniones += 1
                df_merged = df_merged.rename(columns={'ESTACION': 'ESTACION_CA'})
                columnas_finales = ['ESTACION_CA', 'MAGNITUD', 'ANO', 'MES', 'DIA', 'HORA', 'VALOR_HORA']
                columnas_meteo = [col for col in df_merged.columns if 'VALOR_HORA_' in col]
                df_final = df_merged[columnas_finales + columnas_meteo]
                datasets_combinados.append(df_final)

df_dataset_completo = pd.concat(datasets_combinados, ignore_index=True)


os.makedirs("/content/DatasetCompleto", exist_ok=True)
df_dataset_completo.to_csv("/content/DatasetCompleto/dataset_combinado_final.csv", index=False)

print(f"Combinación completa. Total combinaciones exitosas: {contador_uniones}")
print(f"Dataset final guardado con {len(df_dataset_completo)} registros.")


Ahora vamos a ver cuantos datos VALOR_HORA ahi en el dataset fonal para ver que seguimos manteniendo el mismo numero de datos

In [ ]:
total_valores_hora = df_dataset_completo['VALOR_HORA'].notna().sum()
print(f"Total de valores de VALOR_HORA en el dataset final: {total_valores_hora}")


Ahora vamos a ver cuantos valores nulos hay

In [ ]:
total_nulos = df_dataset_completo.isna().sum().sum()
print(f"Total de valores nulos en el dataset final: {total_nulos}")

No hay nulos, el dataset está listo para entrenar

In [ ]:
nulos = df_dataset_completo.isna().sum()
totales = df_dataset_completo.shape[0]

resumen_nulos = pd.DataFrame({
    'Nulos': nulos,
    'Total': totales,
    'Porcentaje_nulos (%)': (nulos / totales * 100).round(2)
})

resumen_nulos = resumen_nulos.sort_values(by='Nulos', ascending=False)

print(" Nulos, total y porcentaje por columna:")
print(resumen_nulos)


Veamos más información del dataset

In [ ]:
df_dataset_completo.info()
df_dataset_completo.describe().T

Ahora, realizaremos one-hot-encoding para las variables categóricas (ESTACION_CA Y MAGNITUD)

In [ ]:
ruta_entrada = "/content/DatasetCompleto/dataset_combinado_final.csv"
df = pd.read_csv(ruta_entrada)


df_encoded = pd.get_dummies(df, columns=["ESTACION_CA", "MAGNITUD"], prefix=["ESTACION_CA", "MAGNITUD"])

carpeta_salida = "/content/DATASETFINAL_1_TRANSFORMACION"
os.makedirs(carpeta_salida, exist_ok=True)
ruta_salida = os.path.join(carpeta_salida, "dataset_final_encoded.csv")
df_encoded.to_csv(ruta_salida, index=False)

print(f"Dataset con one-hot encoding guardado en: {ruta_salida}")


Vamos comprobando

In [ ]:
df_encoded = pd.read_csv('/content/DATASETFINAL_1_TRANSFORMACION/dataset_final_encoded.csv')

total_valores_post_encoding = df_encoded['VALOR_HORA'].notna().sum()

print(f"Total de valores de VALOR_HORA después del one-hot encoding: {total_valores_post_encoding}")


In [ ]:
df_encoded.info()

In [ ]:
nulos = df_encoded.isna().sum()
totales = df_encoded.shape[0]

resumen_nulos = pd.DataFrame({
    'Nulos': nulos,
    'Total': totales,
    'Porcentaje_nulos (%)': (nulos / totales * 100).round(2)
})

resumen_nulos = resumen_nulos.sort_values(by='Nulos', ascending=False)

print(" Nulos, total y porcentaje por columna:")
print(resumen_nulos)

Ahora transformaremos las variables temporales en variables cíclicas

In [ ]:
import numpy as np
import pandas as pd
import os

ruta_entrada = "/content/DATASETFINAL_1_TRANSFORMACION/dataset_final_encoded.csv"
df = pd.read_csv(ruta_entrada)

df['HORA_sin'] = np.sin(2 * np.pi * df['HORA'] / 24)
df['HORA_cos'] = np.cos(2 * np.pi * df['HORA'] / 24)

df['DIA_sin'] = np.sin(2 * np.pi * df['DIA'] / 31)
df['DIA_cos'] = np.cos(2 * np.pi * df['DIA'] / 31)

df['MES_sin'] = np.sin(2 * np.pi * df['MES'] / 12)
df['MES_cos'] = np.cos(2 * np.pi * df['MES'] / 12)

df['AÑO_sin'] = np.sin(2 * np.pi * (df['ANO'] - 2019) / 6)
df['AÑO_cos'] = np.cos(2 * np.pi * (df['ANO'] - 2019) / 6)

df = df.drop(columns=['HORA', 'DIA', 'MES', 'ANO'])

carpeta_salida = "/content/DATASETFINAL_2_TRANSFORMACION"
os.makedirs(carpeta_salida, exist_ok=True)
ruta_salida = os.path.join(carpeta_salida, "dataset_final_ciclico.csv")
df.to_csv(ruta_salida, index=False)

print(f"Dataset con variables cíclicas guardado en: {ruta_salida}")


Comprobamos

In [ ]:
df_ciclico = pd.read_csv('/content/DATASETFINAL_2_TRANSFORMACION/dataset_final_ciclico.csv')
total_valores = df_ciclico['VALOR_HORA'].notna().sum()

print(f"Total de valores de VALOR_HORA : {total_valores}")


In [ ]:
df_ciclico.info()

In [ ]:
nulos = df_ciclico.isna().sum()
totales = df_ciclico.shape[0]

resumen_nulos = pd.DataFrame({
    'Nulos': nulos,
    'Total': totales,
    'Porcentaje_nulos (%)': (nulos / totales * 100).round(2)
})

resumen_nulos = resumen_nulos.sort_values(by='Nulos', ascending=False)

print(" Nulos, total y porcentaje por columna:")
print(resumen_nulos)

Genial, ahora haremos la normalización de datos numéricos , el objetivo es escalar todas las variables numéricas a un rango [0.1] de forma que dejemos ya todas las variables listas para entrenar modelos de Machine Learning, se normalizan :

VALOR_HORA
VALOR_HORA_81
VALOR_HORA_82
VALOR_HORA_83
VALOR_HORA_86
VALOR_HORA_87
VALOR_HORA_88
VALOR_HORA_89
HORA_sin, HORA_cos
DIA_sin, DIA_cos
MES_sin, MES_cos
AÑO_sin, AÑO_cos

Guardaremos el scaler

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import joblib

ruta_entrada = "/content/DATASETFINAL_2_TRANSFORMACION/dataset_final_ciclico.csv"
df = pd.read_csv(ruta_entrada)


columnas_a_normalizar = [
    col for col in df.columns
    if col.startswith("VALOR_HORA") or col.endswith("_sin") or col.endswith("_cos")
]


scaler = MinMaxScaler()
df[columnas_a_normalizar] = scaler.fit_transform(df[columnas_a_normalizar])

carpeta_salida = "/content/DATASETFINAL_3_TRANSFORMACION"
os.makedirs(carpeta_salida, exist_ok=True)
ruta_salida = os.path.join(carpeta_salida, "dataset_final_normalizado.csv")
df.to_csv(ruta_salida, index=False)


joblib.dump(scaler, '/content/scaler_magnitudes.pkl')
print("Scaler guardado como 'scaler_magnitudes.pkl'")


print(f"Dataset normalizado guardado en: {ruta_salida}")


Veamos la información del dataset final

In [ ]:
df_normalizado = pd.read_csv("/content/DATASETFINAL_3_TRANSFORMACION/dataset_final_normalizado.csv")
print(f"Número total de filas: {len(df_normalizado)}")
columnas_a_comprobar = [
    col for col in df_normalizado.columns
    if col.startswith("VALOR_HORA") or col.endswith("_sin") or col.endswith("_cos")
]

for col in columnas_a_comprobar:
    min_val = df_normalizado[col].min()
    max_val = df_normalizado[col].max()
    if min_val < 0 or max_val > 1:
        print(f"{col} fuera del rango: min={min_val}, max={max_val}")

Vemos que en la normalización hay un valor de la magnitud 83 que se ha salido del rango pero es tan tan tan pequeño, que esto no marcará ningun tipo de diferencia, lo más probable es que se deba simplemente a una imprecisión por coma flotante

In [ ]:
import pandas as pd
df_normalizado = pd.read_csv('/content/DATASETFINAL_3_TRANSFORMACION/dataset_final_normalizado.csv')
total_valores = df_normalizado['VALOR_HORA'].notna().sum()

print(f"Total de valores de VALOR_HORA : {total_valores}")

In [ ]:
nulos = df_normalizado.isna().sum()
totales = df_normalizado.shape[0]

resumen_nulos = pd.DataFrame({
    'Nulos': nulos,
    'Total': totales,
    'Porcentaje_nulos (%)': (nulos / totales * 100).round(2)
})

resumen_nulos = resumen_nulos.sort_values(by='Nulos', ascending=False)

print(" Nulos, total y porcentaje por columna:")
print(resumen_nulos)

A continuación entrenaremos el modelo en local, para ello he utilizado pytorch, me instalé pytorch y tambien me instalé cuda, la versión compatible con la versión que me había instalado de pytorch, esto me permitió utilizar mi gráfica de NVIDIA para entrenar el modelo lo cual me permitiría muchos mejores resultados,en mi caso, CUDA 11.8 era compatible con mi GPU y versión de drivers.Para aislar dependencias, utilicé un entorno virtual y en este entorno virtual ejecute mi entrenamiento: